In [ ]:
import numpy as np
from collections import Counter, defaultdict
import concurrent.futures

# Define ranks and suits
ranks = np.arange(13)  # Map ranks to integers 0-12 (2 to Ace)
suits = np.arange(4)   # Map suits to integers 0-3 (hearts, diamonds, clubs, spades)

# Create a deck with two identical sets of 52 cards
deck = np.array([(rank, suit) for rank in ranks for suit in suits] * 2, dtype=[('rank', 'i4'), ('suit', 'i4')])

def print_hand(hand):
    """Print the hand in a readable 'rank of suit' format."""
    hand_cards = []
    for card in hand:
        rank = card['rank']
        suit = card['suit']
        suit_name = ["Hearts", "Diamonds", "Clubs", "Spades"][suit]
        rank_name = ["2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K", "A"][rank]
        hand_cards.append(f"{rank_name} of {suit_name}")
    print("Hand:", ", ".join(hand_cards))

def deal_hand(deck):
    """Shuffle the deck and deal the first 5 cards for a single trial."""
    shuffled_deck = np.random.permutation(deck)  # Shuffle the entire deck randomly
    hand = shuffled_deck[:5]  # Take the first 5 cards after shuffling
    return shuffled_deck, hand

def check_flush_case(hand):
    """Classify a Flush hand into Case 0, Case 1, or Case 2, or eliminate Royal/Straight Flush."""
    # Ensure all cards are of the same suit
    if len(set(card['suit'] for card in hand)) != 1:
        return -1  # Not a Flush

    # Extract ranks and sort them
    ranks = sorted(card['rank'] for card in hand)
    rank_counts = Counter(ranks)

    # Eliminate Royal Flush and Straight Flush
    if (max(ranks) - min(ranks) == 4) and (len(rank_counts) == 5):
        return -1  # This is a Straight Flush

    # Case 0: Five cards of the same suit with all unique ranks, excluding Straight Flush
    if len(rank_counts) == 5:
        return 0

    # Case 1: Five cards of the same suit, with a pair and three unique ranks
    if len(rank_counts) == 4 and any(count == 2 for count in rank_counts.values()):
        return 1

    # Case 2: Five cards of the same suit, with two pairs and one unique card
    if len(rank_counts) == 3 and sorted(rank_counts.values()) == [1, 2, 2]:
        return 2

    return -1  # If it doesn't match any specific case

def run_simulation(num_trials):
    """Run the simulation, count occurrences of each case, and collect examples as they occur."""
    flush_count = 0
    counts = Counter()
    local_examples = defaultdict(list)

    for _ in range(num_trials):
        _, hand = deal_hand(deck)  # Shuffle and deal a fresh hand for each trial
        case = check_flush_case(hand)

        if case != -1:  # It's a valid Flush case (0, 1, or 2)
            flush_count += 1
            counts[case] += 1
            # Collect up to 10 examples for each case if not already filled
            if case in [0, 1] and len(local_examples[case]) < 10:
                local_examples[case].append(hand)
            elif case == 2:  # Collect all examples for Case 2
                local_examples[case].append(hand)

    return flush_count, counts, local_examples

def run_simulation_parallel(trials, num_workers=4):
    """Run the simulation in parallel using multiple workers."""
    chunk_size = trials // num_workers
    total_flush_count = 0
    example_hands = defaultdict(list)  # Store examples globally after combining
    combined_counts = Counter()

    with concurrent.futures.ProcessPoolExecutor(max_workers=num_workers) as executor:
        futures = [executor.submit(run_simulation, chunk_size) for _ in range(num_workers)]
        results = [f.result() for f in concurrent.futures.as_completed(futures)]

    # Aggregate results from all workers
    for flush_count, counts, local_examples in results:
        total_flush_count += flush_count
        combined_counts.update(counts)
        # Merge examples up to 10 per case, but all examples for Case 2
        for case, hands in local_examples.items():
            if case == 2:
                example_hands[case].extend(hands)  # Collect all examples for Case 2
            elif len(example_hands[case]) < 10:
                example_hands[case].extend(hands[:10 - len(example_hands[case])])

    return total_flush_count, combined_counts, example_hands

def calculate_flush_probability(flush_count, total_hands):
    """Calculate the probability of getting a Flush."""
    return flush_count / total_hands

def main():
    # Part 1: Print the first 100 shuffled decks and dealt hands for verification
    print("\nRunning 100 trials with shuffled deck:")
    run_simulation(100)  # Running 100 trials with output formatting

    # Part 2: Run large simulations for specified trial counts
    trial_counts = [1_000_000, 10_000_000, 100_000_000]
    for trials in trial_counts:
        print(f"\nRunning parallel simulation with {trials} trials...")
        flush_count, counts, example_hands = run_simulation_parallel(trials)

        # Calculate Flush probability
        flush_probability = calculate_flush_probability(flush_count, trials)

        # Print results and Flush probability
        print(f"\nResults for {trials} trials:")
        print(f"Total Flush occurrences: {flush_count}")
        print(f"Flush probability: {flush_probability:.8f}")
        print(f"Case 0: {counts[0]} occurrences")
        print(f"Case 1: {counts[1]} occurrences")
        print(f"Case 2: {counts[2]} occurrences")

        # After 1 million trials, print 10 example hands for Case 0, 10 for Case 1, and all for Case 2
        if trials == 1_000_000:
            print("\n10 Example Hands for Case 0:")
            for hand in example_hands[0][:10]:  # Print up to 10 hands for Case 0
                print_hand(hand)

            print("\n10 Example Hands for Case 1:")
            for hand in example_hands[1][:10]:  # Print up to 10 hands for Case 1
                print_hand(hand)

            print("\nAll Case 2 Examples:")
            for hand in example_hands[2]:  # Print all hands for Case 2
                print_hand(hand)

if __name__ == "__main__":
    main()



Running 100 trials with shuffled deck:

Running parallel simulation with 1000000 trials...

Results for 1000000 trials:
Total Flush occurrences: 2868
Flush probability: 0.00286800
Case 0: 1792 occurrences
Case 1: 1004 occurrences
Case 2: 72 occurrences

10 Example Hands for Case 0:
Hand: 9 of Spades, 5 of Spades, 10 of Spades, 3 of Spades, 7 of Spades
Hand: 9 of Diamonds, Q of Diamonds, K of Diamonds, 2 of Diamonds, A of Diamonds
Hand: K of Clubs, 4 of Clubs, 6 of Clubs, 5 of Clubs, J of Clubs
Hand: 6 of Spades, A of Spades, 8 of Spades, 7 of Spades, 9 of Spades
Hand: K of Hearts, J of Hearts, 8 of Hearts, Q of Hearts, 2 of Hearts
Hand: 6 of Clubs, 3 of Clubs, A of Clubs, 7 of Clubs, Q of Clubs
Hand: 10 of Clubs, 3 of Clubs, 4 of Clubs, 6 of Clubs, 9 of Clubs
Hand: 3 of Clubs, 8 of Clubs, J of Clubs, 2 of Clubs, 7 of Clubs
Hand: A of Spades, 10 of Spades, 2 of Spades, 8 of Spades, 7 of Spades
Hand: Q of Hearts, A of Hearts, J of Hearts, 2 of Hearts, 4 of Hearts

10 Example Hands for C

In [ ]:
import numpy as np
from collections import Counter, defaultdict
import concurrent.futures

# Define ranks and suits
ranks = np.arange(13)  # Map ranks to integers 0-12 (2 to Ace)
suits = np.arange(4)   # Map suits to integers 0-3 (hearts, diamonds, clubs, spades)

# Create a deck with two identical sets of 52 cards
deck = np.array([(rank, suit) for rank in ranks for suit in suits] * 2, dtype=[('rank', 'i4'), ('suit', 'i4')])

def print_hand(hand):
    """Print the hand in a readable 'rank of suit' format."""
    hand_cards = []
    for card in hand:
        rank = card['rank']
        suit = card['suit']
        suit_name = ["Hearts", "Diamonds", "Clubs", "Spades"][suit]
        rank_name = ["2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K", "A"][rank]
        hand_cards.append(f"{rank_name} of {suit_name}")
    print("Hand:", ", ".join(hand_cards))

def deal_hand(deck):
    """Shuffle the deck and deal the first 5 cards for a single trial."""
    shuffled_deck = np.random.permutation(deck)  # Shuffle the entire deck randomly
    hand = shuffled_deck[:5]  # Take the first 5 cards after shuffling
    return shuffled_deck, hand

def check_flush_case(hand):
    """Classify a Flush hand into Case 0, Case 1, or Case 2, or eliminate Royal/Straight Flush."""
    # Ensure all cards are of the same suit
    if len(set(card['suit'] for card in hand)) != 1:
        return -1  # Not a Flush

    # Extract ranks and sort them
    ranks = sorted(card['rank'] for card in hand)
    rank_counts = Counter(ranks)

    # Eliminate Royal Flush and Straight Flush
    if (max(ranks) - min(ranks) == 4) and (len(rank_counts) == 5):
        return -1  # This is a Straight Flush

    # Case 0: Five cards of the same suit with all unique ranks, excluding Straight Flush
    if len(rank_counts) == 5:
        return 0

    # Case 1: Five cards of the same suit, with a pair and three unique ranks
    if len(rank_counts) == 4 and any(count == 2 for count in rank_counts.values()):
        return 1

    # Case 2: Five cards of the same suit, with two pairs and one unique card
    if len(rank_counts) == 3 and sorted(rank_counts.values()) == [1, 2, 2]:
        return 2

    return -1  # If it doesn't match any specific case


def run_simulation(num_trials):
    """Run the simulation, count occurrences of each case, and collect examples as they occur."""
    flush_count = 0
    counts = Counter()
    local_examples = defaultdict(list)

    for _ in range(num_trials):
        _, hand = deal_hand(deck)  # Shuffle and deal a fresh hand for each trial
        case = check_flush_case(hand)

        if case != -1:  # It's a valid Flush case (0, 1, or 2)
            flush_count += 1
            counts[case] += 1
            # Collect up to 10 examples for Case 0 and Case 1
            if case in [0, 1] and len(local_examples[case]) < 10:
                local_examples[case].append(hand)

    return flush_count, counts, local_examples

def run_simulation_parallel(trials, num_workers=4):
    """Run the simulation in parallel using multiple workers."""
    chunk_size = trials // num_workers
    total_flush_count = 0
    example_hands = defaultdict(list)  # Store examples globally after combining
    combined_counts = Counter()

    with concurrent.futures.ProcessPoolExecutor(max_workers=num_workers) as executor:
        futures = [executor.submit(run_simulation, chunk_size) for _ in range(num_workers)]
        results = [f.result() for f in concurrent.futures.as_completed(futures)]

    # Aggregate results from all workers
    for flush_count, counts, local_examples in results:
        total_flush_count += flush_count
        combined_counts.update(counts)
        # Merge examples up to 10 per case
        for case, hands in local_examples.items():
            if len(example_hands[case]) < 10:
                example_hands[case].extend(hands[:10 - len(example_hands[case])])

    return total_flush_count, combined_counts, example_hands

def calculate_flush_probability(flush_count, total_hands):
    """Calculate the probability of getting a Flush."""
    return flush_count / total_hands

def main():
    # Part 1: Print the first 100 shuffled decks and dealt hands for verification
    print("\nFirst 100 shuffles and hands:")
    for trial in range(100):
        shuffled_deck, hand = deal_hand(deck)
        print(f"\nShuffle {trial + 1}:")
        print_hand(hand)

    # Part 2: Run large simulations for specified trial counts
    trial_counts = [1_00, 1_000,1_000_0, 10_000_0]
    for trials in trial_counts:
        print(f"\nRunning parallel simulation with {trials} trials...")
        flush_count, counts, example_hands = run_simulation_parallel(trials)

        # Calculate Flush probability
        flush_probability = calculate_flush_probability(flush_count, trials)

        # Print results and Flush probability
        print(f"Results for {trials} trials:")
        print(f"Total Flush occurrences: {flush_count}")
        print(f"Flush probability: {flush_probability:.8f}")



if __name__ == "__main__":
    main()



First 100 shuffles and hands:

Shuffle 1:
Hand: 3 of Diamonds, 3 of Spades, 6 of Hearts, 8 of Hearts, Q of Clubs

Shuffle 2:
Hand: 10 of Clubs, K of Hearts, 5 of Hearts, Q of Spades, 6 of Clubs

Shuffle 3:
Hand: K of Hearts, 10 of Spades, 6 of Spades, 4 of Diamonds, 8 of Hearts

Shuffle 4:
Hand: 6 of Diamonds, 5 of Clubs, 10 of Diamonds, 6 of Spades, 5 of Hearts

Shuffle 5:
Hand: 5 of Clubs, 3 of Clubs, 9 of Hearts, 8 of Spades, A of Diamonds

Shuffle 6:
Hand: A of Diamonds, 10 of Diamonds, 8 of Hearts, 2 of Clubs, 6 of Hearts

Shuffle 7:
Hand: 8 of Hearts, 6 of Diamonds, 4 of Diamonds, A of Diamonds, 2 of Hearts

Shuffle 8:
Hand: K of Spades, 2 of Spades, 10 of Diamonds, 6 of Hearts, K of Clubs

Shuffle 9:
Hand: A of Hearts, Q of Spades, 4 of Diamonds, Q of Spades, A of Diamonds

Shuffle 10:
Hand: J of Clubs, A of Clubs, 5 of Hearts, 6 of Clubs, 7 of Hearts

Shuffle 11:
Hand: 5 of Spades, Q of Spades, 3 of Hearts, J of Diamonds, 9 of Diamonds

Shuffle 12:
Hand: K of Hearts, 6 of Diam